In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='US/Eastern')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.start_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["start_station_id", "start_hour"]).reset_index(drop=True)
ts_data["start_hour"] = ts_data["start_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-05-09 01:25:27,453 INFO: Initializing external client
2025-05-09 01:25:27,455 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:25:29,197 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Fetching data from 2025-04-10 01:25:27.446566-04:00 to 2025-05-09 00:25:27.446566-04:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


In [9]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-10 10:29:23,282 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:29:23,413 INFO: Initializing external client
2025-05-10 10:29:23,415 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:29:25,430 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sahit\\AppData\\Local\\Temp\\d879112b-8ab3-4d3e-95e8-05135e06c205\\citibike_demand_predictor\\4\\lgb_model.pkl'

In [5]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [6]:
predictions["start_hour"] = current_date.ceil('h')
predictions

,start_station_id,predicted_demand,start_hour
0,5308.04,9.0,2025-05-09 02:00:00-04:00
1,6230.02,5.0,2025-05-09 02:00:00-04:00
2,5872.10,5.0,2025-05-09 02:00:00-04:00


In [7]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["start_station_id", "start_hour"],
    event_time="start_hour",
)

2025-05-09 01:26:55,740 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:26:55,757 INFO: Initializing external client
2025-05-09 01:26:55,765 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:26:57,051 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682


In [8]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213682/fs/1202282/fg/1454470


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 3/3 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: citibike_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213682/jobs/named/citibike_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('citibike_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)